In [3]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

## Last Preprocessing
Rename the columns as well as change positive tweets from target = 4 to target = 1

In [4]:
df = pd.read_csv("../model-files/tweets-cleaned.csv", encoding='ISO-8859-1')
df = df.iloc[:, [0, 2]] # Don't want the data from before processing
df

/var/folders/wh/j0mpnsjn5dn3d140byhbb9h80000gn/T/ipykernel_23981/2755469249.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../model-files/tweets-cleaned.csv", encoding='ISO-8859-1')


,target,clean
0,0,URL aww bummer shoulda got david carr third day
1,0,upset update facebook texting might cry result...
2,0,dived many time ball managed save rest go bound
3,0,whole body feel itchy like fire
4,0,behaving mad see
...,...,...
1700025,4,chris great hear due time amp reminder indeed ...
1700026,4,thanks shout great aboard
1700027,4,hey long URL talk
1700028,4,matt would say URL adulthood URL


In [3]:
df['target'].replace(4, 1, inplace=True) # Replace all the 4s with 1s
df.rename(columns={'target': 'sentiment', 'text':'tweet'}, inplace=True) # Rename the columns
df

,sentiment,clean
0,0,URL aww bummer shoulda got david carr third day
1,0,upset update facebook texting might cry result...
2,0,dived many time ball managed save rest go bound
3,0,whole body feel itchy like fire
4,0,behaving mad see
...,...,...
1700025,4,chris great hear due time amp reminder indeed ...
1700026,4,thanks shout great aboard
1700027,4,hey long URL talk
1700028,4,matt would say URL adulthood URL


In [4]:
df.dtypes

sentiment    object
clean        object
dtype: object

In [55]:
text, sentiment = list(df['clean']), list(df['sentiment']) # Turn these both into lists

text = [str(item) for item in text] 
positive_text = text[850000:] # Dataset arranged in a way such that the first half is negative and second half is positive
negative_text = text[:850000]
positive_sentiment = sentiment[850000:]
negative_sentiment = sentiment[:850000]
negative_text[0].split()

['URL', 'aww', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day']

## Use One-Hot-Encoding

We will be considering the top X frequent Tweets for both categories and doing one-hot-encoding with them. Basically, it will be a matrix where each row represents a message and each column represents each X most frequent words. If that particular word is present, mark it as 1. Otherwise, words that aren't in that message are marked with 0

## DONT RUN AGAIN AFTER WE PICKLED THE VECTORS

#### Count the frequencies up

In [5]:
with open("../model-files/stop-words.txt") as f:
    stopwords = [line.strip() for line in f]
stopwords = set(stopwords) # Set of stop words to check against for the clean method for each prediction, keep in mind this model takes in cleaned data so should only be used for prediction


negative_word_count = 0 # Total number of words for each class
positive_word_count = 0
positive_words = {}  # Word frequencies for each class
negative_words = {}

In [20]:
for i in range(len(positive_text)):
    splitted = positive_text[i].split()
    for j in range(len(splitted)):
        if splitted[j] not in positive_words: # Keep track of 
            positive_words[splitted[j]] = 0
        positive_words[splitted[j]] += 1
        positive_word_count += 1

for i in range(len(negative_text)):
    splitted = negative_text[i].split()
    for j in range(len(splitted)):
        if splitted[j] not in negative_words:
            negative_words[splitted[j]] = 0
        negative_words[splitted[j]] += 1
        negative_word_count += 1

#### Grab the top 1000 most frequent words with significant meaning

In [56]:
most_freq_pos = list()
for k, v in positive_words.items():
    most_freq_pos.append((v, k))
most_freq_neg = list()
for k, v in negative_words.items():
    most_freq_neg.append((v, k))

most_freq_pos.sort()
most_freq_neg.sort() 
# We're gonna take top 1000 unique words, but if the lists have the same word at the same index, don't use it
print(most_freq_pos[-10:])
print(most_freq_neg[-10:])

[(64292, 'got'), (73334, 'lol'), (77178, 'thanks'), (78728, 'get'), (80260, 'like'), (90660, 'quot'), (106220, 'love'), (118700, 'day'), (131674, 'good'), (528914, 'URL')]
[(69096, 'going'), (71748, 'want'), (78616, 'got'), (78866, 'today'), (86172, 'like'), (95790, 'work'), (98176, 'get'), (99002, 'go'), (107166, 'day'), (470392, 'URL')]


In [50]:
pos_index = len(most_freq_pos) - 1
res = set()
for neg_index in range(len(most_freq_neg)-1, -1, -1):
    if most_freq_pos[pos_index][1] != most_freq_neg[neg_index][1]:
        if len(res) == 999: # Only add one if we are getting close, positive has more words so use that
            res.add(most_freq_pos[pos_index][1])
        else:
            res.add(most_freq_pos[pos_index][1])
            res.add(most_freq_neg[neg_index][1])
    pos_index -= 1
    if len(res) == 1000:
        break
print(len(res))
print(res)

with open("../model-files/words.txt", 'w') as file:
    for word in res:
        file.write(word + '\n')

1000
{'yummy', 'crazy', 'making', 'cuz', 'suck', 'say', 'facebook', 'buddy', 'cool', 'screen', 'mail', 'worst', 'starting', 'hahaha', 'st', 'read', 'hopefully', 'care', 'awake', 'house', 'yay', 'moon', 'pc', 'awful', 'yes', 'minute', 'stuck', 'lakers', 'men', 'sit', 'understand', 'doesnt', 'perfect', 'drive', 'seeing', 'call', 'learn', 'blog', 'shift', 'photo', 'ago', 'plan', 'anything', 'people', 'thanks', 'stop', 'fast', 'ipod', 'back', 'reason', 'fuck', 'soo', 'thinking', 'even', 'coming', 'drunk', 'rain', 'drink', 'track', 'working', 'btw', 'cd', 'everyone', 'dance', 'bro', 'gutted', 'youu', 'said', 'air', 'eat', 'else', 'nap', 'terrible', 'felt', 'shoot', 'tomorrow', 'friend', 'change', 'wit', 'thats', 'worth', 'well', 'aww', 'ya', 'twitter', 'full', 'friday', 'went', 'needed', 'dad', 'used', 'chocolate', 'nd', 'answer', 'haha', 'soon', 'huge', 'shopping', 'scary', 'fact', 'date', 'holiday', 'stomach', 'sunday', 'yum', 'enjoying', 'min', 'moving', 'tom', 'studying', 'annoying', 'i

#### Make a new dataframe with the hot encodings for each message

In [1]:
word_list = []
with open("../model-files/words.txt", 'r') as file:
    for line in file:
        word_list.append(line.strip())# Order doesn't matter, but needs to stay consistent so convert the set to a list
word_list_dict = dict()
for i in range(len(word_list)): # Map words to their index to make the next part O(1) instead of O(N)
    word_list_dict[word_list[i]] = i

hot_encode = dict()
vectors = list()
for message in text:
    tokens = message.split()
    vector = [0] * len(word_list)
    for token in tokens:
        if token in word_list_dict:
            vector[word_list_dict[token]] = 1 # We have this token so for this vector, mark it accordingly
    vectors.append(vector)

# So now each index of vectors's output is just the corresponding index in the list called "sentiment"

NameError: name 'text' is not defined

In [ ]:
import pickle
with open('model-files/training_vectors.pkl', 'wb') as f:
    pickle.dump(vectors, f)